In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

De início, realizei uma análise no arquivo excel, e fiz uma série de pré-processamentos direto no arquivo.

1) Das 5644 linhas, exclui todas as que tinham todas as colunas de atributos (variáveis independentes ou features) sem valores. A base foi reduzida a 623 linhas.

2) Em seguida, excluí todas as colunas de atributos que não tinham valor para nenhuma linha, e (por enquanto) também as colunas categóricas. A base foi reduzida a 68 colunas.

3) Coluna "SARS-Cov-2 exam result" teve os valores alterados : "negative" --> 0, "positive"--> 1. São 87 positivos na base reduzida.

4) Portanto, ficamos com as colunas: SARS-Cov-2 exam result,	Patient addmited to regular ward, Patient addmited to semi-intensive unit, e Patient addmited to intensive care unit como labels (variáveis dependentes). Nesta primeira análise somente a primeira será utilizada.

5) As demais colunas serão os atributos. Patient ID não será utilizado.

6) Células vazias (nan) foram preenchidas com zero



### Carga do novo arquivo reduzido

In [ ]:
ds_red = pd.read_excel('/kaggle/input/newfile/dataset_modificado.xlsx', index_col=0)

In [ ]:
ds_red

In [ ]:
ds_red.shape

In [ ]:
ds_red.describe

Duas análises são necessárias:

1) a previsão de diagnóstico positivo/negativo para o vírus (coluna SARS-Cov-2 exam result). Aqui temos como saída apenas uma coluna categórica.

2) a previsão de qual será o encaminhamento para o paciente (colunas Patient addmited to regular ward, Patient addmited to semi-intensive unit, e Patient addmited to intensive care unit). Aqui, então, temos 3 colunas categóricas.

3) o modelo de classificação escolhido foi o de ÁRVORES DE DECISÃO, depois comparado com FLORESTAS ALEATÓRIAS

# ANÁLISE DE PREVISÃO DE DIAGNÓSTICO

In [ ]:
import numpy as np
from sklearn.tree import DecisionTreeClassifier


Divisão da massa de dados em pacientes (linhas) de TREINAMENTO e TESTE

In [ ]:
def dividir_treino_teste(data, perc_treinamento):
  np.random.seed(0)
  #np.random.shuffle(data)
  divisao = int(data.shape[0] * perc_treinamento)
  conj_treinamento = data[:divisao]
  conj_teste = data[divisao:]
  
  return conj_treinamento,conj_teste

Divisão das colunas em atributos (variáveis independentes) e labels (rótulos, variável dependente). Também **normaliza** os atributos.

In [ ]:
def gerar_atributos_labels(data):
  
  labels = data['SARS-Cov-2 exam result']

  atributos = data.iloc[:,5:]

  atributos= preprocessing.StandardScaler().fit(atributos).transform(atributos)

  return atributos, labels

## ÁRVORE DE DECISÃO

Criação do modelo

In [ ]:
def decision_tree_previsto_real(data):
    
  # dividir o conjunto de dados em treinamento (70%) e teste
  pac_treinamento, pac_teste = dividir_treino_teste(data, 0.7)

  # gerar os conjuntos de atributos e labels para treinamento e testes
  atributos_treinamento, labels_treinamento = gerar_atributos_labels(pac_treinamento)
  atributos_teste, labels_teste = gerar_atributos_labels(pac_teste)

  # cria uma instância de classificador por árvore de decisão
  dtr = DecisionTreeClassifier()

  # treina o classificador com os pacientes de treinamento
  dtr.fit(atributos_treinamento,labels_treinamento)

  # obtém previsões para os atributos de teste
  previsoes = dtr.predict(atributos_teste)

  # retorna as previsões e os labels teste (reais)
  return previsoes,labels_teste




Análise da saída do modelo gerado

In [ ]:
classe_prevista, classe_real = decision_tree_previsto_real(ds_red)

# mostra os 10 primeiros resultados
print("Alguns resultados iniciais...\n   previsto,  real")
for i in range(10):
    print("{}. {}, {}".format(i, classe_prevista[i], classe_real[i]))
 

In [ ]:
import numpy as np
from matplotlib import pyplot as plt
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import cross_val_predict
from sklearn.tree import DecisionTreeClassifier
#from support_functions import plot_confusion_matrix




Cálculo da precisão (accuracy) do modelo, e matriz de confusão

In [ ]:
# Implement the following function
def calcula_precisao(previsto, real):
  previsoes = 0
  previsoes_corretas = 0
  for i in range(len(previsto)):
    previsoes += 1
    if(previsto[i] == real[i]):
      previsoes_corretas += 1
  return (previsoes_corretas / previsoes)
    


In [ ]:
# split the data
atributos, labels = gerar_atributos_labels(ds_red)

# train the model to get predicted and actual classes
dtc = DecisionTreeClassifier()
previsto = cross_val_predict(dtc, atributos, labels, cv=10)

# calculate the model score using your function
model_score = calcula_precisao(previsto, labels)
print("Score de precisão:", model_score)

# calculate the models confusion matrix using sklearns confusion_matrix function
class_labels = list(set(labels))
model_cm = confusion_matrix(y_true=labels, y_pred=previsto, labels=class_labels)

model_cm

## FLORESTA ALEATÓRIA

In [ ]:
import numpy as np
from matplotlib import pyplot as plt
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import cross_val_predict
from sklearn.ensemble import RandomForestClassifier


def random_forest_previsto_real(data, n_estimators):
  
  # gerar atributos e labels
  atributos, labels = gerar_atributos_labels(data)

  # instanciar um classificador por floresta aleatória
  rfc = RandomForestClassifier(n_estimators=n_estimators)
  
  # obtem previsoes usando 10-fold cross validation
  previsto = cross_val_predict(rfc,atributos,labels,cv=10)

  # retorna valores previstos e reais
  return previsto,labels


# get the predicted and actual classes
num_arvores = 50              
previsto, real = random_forest_previsto_real(ds_red, num_arvores)

# calculate the model score using your function
precisao = calcula_precisao(previsto, real)
print("Score de precisão:", precisao)

# calculate the models confusion matrix using sklearns confusion_matrix function
class_labels = list(set(real))
model_cm = confusion_matrix(y_true=real, y_pred=previsto, labels=class_labels)

model_cm


Portanto, dos dois modelos acima, o de floresta aleatória chega a uma melhor precisão para diagnóstico positivo/negativo.

# ANÁLISE DE ENCAMINHAMENTO

Para esta análise, foi gerada uma cópia do dataset utilizado acima, onde as colunas: "Patient addmited to regular ward", "Patient addmited to semi-intensive unit", e "Patient addmited to intensive care unit" foram substituídas pela coluna "encaminhamento", que possui os valores : 

RW --> Se "Patient addmited to regular ward" = 1

SIU --> Se "Patient addmited to semi-intensive unit" = 1

ICU --> Se "Patient addmited to intensive care unit" = 1

NONE --> Se as três colunas acima = 0

As demais colunas permanecem as mesmas utilizadas na análise inicial.



In [ ]:
import itertools
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import NullFormatter
import pandas as pd
import numpy as np
import matplotlib.ticker as ticker
from sklearn import preprocessing
%matplotlib inline

In [ ]:
ds_enc = pd.read_excel('/kaggle/input/classif/dataset_classif.xlsx', index_col=0)

In [ ]:
ds_enc

In [ ]:
ds_enc.shape

In [ ]:
ds_enc['encaminhamento'].value_counts()

Abaixo uma breve análise gráfica de um atributo exemplo (Hematocrit) observado versus o quantil de idade, mostrando o agrupamento por encaminhamento. É possível ver que os encaminhamentos realizados (diferentes de NONE) concentram-se nas faixas de idade iniciais (bebês e recém nascidos) e idosos.

In [ ]:
import seaborn as sns

bins = np.linspace(ds_enc.Hematocrit.min(), ds_enc.Hematocrit.max(), 10)
g = sns.FacetGrid(ds_enc, col="Patient age quantile", hue="encaminhamento", palette="Set1", col_wrap=2)
g.map(plt.hist, 'Hematocrit', bins=bins, ec="k")

g.axes[-1].legend()
plt.show()

Seleção dos labels e atributos, seguindo o mesmo critério acima.

In [ ]:
atributos = ds_enc.iloc[:,3:]

X = atributos
y = ds_enc['encaminhamento'].values

In [ ]:
X

In [ ]:
y

Normalização dos atributos

In [ ]:
X= preprocessing.StandardScaler().fit(X).transform(X)
X[0:5]

## Classificação

Para a classificação, utilizarei a técnica de clusterização - K Nearest Neighbor(KNN). 
Vou manter os nomes em inglês pois trata-se de código que eu já havia desenvolvido em treinamento.

Em seguida compararei com outras técnicas para ver qual tem a melhor precisão - 

- Decision Tree
- Support Vector Machine
- Logistic Regression

## KNN

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.3, random_state=4) #mantendo proporção de treinamento em 70%
print ('Train set:', X_train.shape,  y_train.shape)
print ('Test set:', X_test.shape,  y_test.shape)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

Iniciarei com 3 clusters; abaixo testaremos a precisão para escolher um valor melhor.

In [ ]:
k = 3 
#Train Model and Predict  
neigh = KNeighborsClassifier(n_neighbors = k).fit(X_train,y_train)
neigh

In [ ]:
yhat = neigh.predict(X_test)
yhat[0:5]

Aqui escolheremos o melhor valor (melhor precisão) para o número de clusters

In [ ]:
lim=25
avg=np.zeros((lim-1))
stdd=np.zeros((lim-1))
cm=[];
maxacc = 0;
bestk = 0;
for k in range(1,lim):
    
    #Train Model and Predict  
    kNN_model = KNeighborsClassifier(n_neighbors=k).fit(X_train,y_train)
    yhat = kNN_model.predict(X_test)
    avg[k-1]=np.mean(yhat==y_test);
    stdd[k-1]=np.std(yhat==y_test)/np.sqrt(yhat.shape[0])
    
    print("k = ",k,"*** accuracy = ",avg[k-1] )
    
    if avg[k-1] > maxacc:
        bestk = k 
        maxacc = avg[k-1]
   
print("The best value of k (best accuracy) is: ",bestk)

Construir novamente o modelo, usando k = 4 para melhor precisão, como obtido acima

In [ ]:
# Building the model again, using the k=7 with best accuracy
from sklearn.neighbors import KNeighborsClassifier
k = 4
#Train Model and Predict  
kNN_model = KNeighborsClassifier(n_neighbors=k).fit(X_train,y_train)
kNN_model

In [ ]:
yhat = kNN_model.predict(X_test)
yhat

## Decision Tree

In [ ]:
from sklearn.tree import DecisionTreeClassifier

In [ ]:
dt = DecisionTreeClassifier(criterion="entropy", max_depth = 4) #profundidade arbitrária por enquanto
dt

In [ ]:
dt.fit(X_train,y_train)

In [ ]:
yhat1 = dt.predict(X_test)
yhat1

Calcular a precisão :

In [ ]:
avg=np.mean(yhat1==y_test);
print("*** accuracy = ",avg )

## Support Vector Machine

In [ ]:
from sklearn import svm
modsvm = svm.SVC()
modsvm.fit(X_train, y_train) 

In [ ]:
yhat2 = modsvm.predict(X_test)
yhat2

Calcular a precisão :

In [ ]:
avg=np.mean(yhat2==y_test);
print("*** accuracy = ",avg )

## Logistic Regression

In [ ]:
from sklearn.linear_model import LogisticRegression
LR = LogisticRegression(C=0.01).fit(X_train,y_train)
LR

In [ ]:
yhat3 = LR.predict(X_test)
yhat3

In [ ]:
avg=np.mean(yhat3==y_test);
print("*** accuracy = ",avg )

Portanto o modelo KNN é o que oferece a melhor precisão.